**Import needed libraries**

In [115]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

import boto3, sagemaker
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer

**Cooling system cleanup**

In [116]:
df_cs = pd.read_excel('Dataset/Cooling+System+Report-06-03-22+14-29 (Masked).xlsx')
df_cs.columns = df_cs.iloc[2]
df_cs = df_cs.iloc[3:]

# only use columns 'LRD', 'SITE TYPE', 'COOLING SYSTEM'
df_cs = df_cs[['LRD', 'SITE TYPE', 'COOLING SYSTEM']]
df_cs.columns = ['LRD', 'CS_Site_Type', 'Cooling_System']
df_cs.reset_index(drop=True, inplace=True)
print(f'Size of Cooling System Dataset: ', df_cs.shape)
df_cs.head(100)

Size of Cooling System Dataset:  (13317, 3)


,LRD,CS_Site_Type,Cooling_System
0,LRD24,Vacant Land,HEX
1,LRD151,Vacant Land,HEX
2,LRD1784,Vacant Land,HEX
3,LRD1784,Vacant Land,HEX
4,LRD1786,Vacant Land,HEX
...,...,...,...
95,LRD1942,Vacant Land,FCU
96,LRD1943,Rooftop,Air Conditioner
97,LRD1943,Rooftop,Air Conditioner
98,LRD1944,Vacant Land,FCU


**Battery data cleanup**

In [117]:
df_dc = pd.read_excel('Dataset/DC+Battery+Report-06-03-22+14-30 (Masked).xlsx')
df_dc.columns = df_dc.iloc[3]
df_dc = df_dc.iloc[4:]

# drop unnecessary columns
df_dc = df_dc[['LRD', 'Administrative Status', 'Battery Brand', 'Battery Capacity (Ah)', 'Total Current Load (Reading at Rectifier controller Display or clamp meter at +ve battery terminal) (Amp)', 'Total bank (Quantity)', 'Battery Technology']]
df_dc.columns = ['LRD', 'DC_Admin_Status', 'DC_Brand', 'DC_Capacity', 'DC_Total_Current_Load', 'Total_Bank', 'DC_Technology']
df_dc.reset_index(drop = True, inplace = True) 

print(f'Size of DC Battery Dataset: ', df_dc.shape)

# Correct the values for 'Administration Status' according to the author's note as stated in the dataset where the Total Current Loads
# that are lesser than 4A will be 'Decomissioned'
df_dc['DC_Total_Current_Load'] = pd.to_numeric(df_dc['DC_Total_Current_Load'], errors='coerce')
df_dc['DC_Admin_Status'] = np.where(df_dc['DC_Total_Current_Load'] > 4, "Active", "Decommissioned")
df_dc.drop('DC_Total_Current_Load', inplace=True, axis=1)

df_dc

Size of DC Battery Dataset:  (10010, 7)


,LRD,DC_Admin_Status,DC_Brand,DC_Capacity,Total_Bank,DC_Technology
0,LRD22703,Decommissioned,Enersys,60,2,Lithium
1,LRD1786,Active,MSB,150,1,VRLA
2,LRD2250,Active,Narada,150,2,VRLA
3,LRD12932,Decommissioned,Enersys,150,1,VRLA
4,LRD2249,Decommissioned,Narada,150,1,VRLA
...,...,...,...,...,...,...
10005,LRD11374,Active,Narada,100,1,Lithium
10006,LRD11406,Decommissioned,Others,150,1,VRLA
10007,LRD11432,Active,Sacredsun,150,1,VRLA
10008,LRD11475,Decommissioned,Sacredsun,150,1,VRLA


**Battery health cleanup**

In [118]:
df_bh = pd.read_excel('Dataset/Battery Health Check (2) (Masked).xlsx')
df_bh.columns = df_bh.iloc[1]
df_bh = df_bh.iloc[2:]

# Drop unnecessary columns
df_bh = df_bh[['Site Name', 'Aging Days', 'Backup Hour']]
df_bh.rename(columns = {'Site Name' : 'LRD'}, inplace = True)
df_bh.dropna(subset = {'Aging Days'}, inplace = True)
df_bh.reset_index(drop = True, inplace = True)

# Calculate the inverse of the power event counts by using the number of events reported
temp = df_bh['LRD'].value_counts()
temp = temp.to_dict()
df_bh['Power Event Count'] = 1/(df_bh['LRD'].map(temp))

df_bh

1,LRD,Aging Days,Backup Hour,Power Event Count
0,LRD2246,1774,>30min to <1hr,0.500000
1,LRD2246,1774,>30min to <1hr,0.500000
2,LRD2274,932,>2hrs to <3hrs,1.000000
3,LRD3701,2105,>3hrs to <4hrs,0.076923
4,LRD3701,2105,>2hrs to <3hrs,0.076923
...,...,...,...,...
2769,LRD3779,480,>2hrs to <3hrs,0.250000
2770,LRD3779,480,>1hr to <2hrs,0.250000
2771,LRD12068,447,>3hrs to <4hrs,0.333333
2772,LRD12068,447,>3hrs to <4hrs,0.333333


**Merge the data to produce a merged-battery-lifespan dataframe**

In [119]:
# merge all the three cleaned dataframes into one
merge_BatteryLifespan = pd.merge(df_bh, df_dc ,how = "inner", on =["LRD"])
merge_BatteryLifespan = pd.merge(merge_BatteryLifespan, df_cs ,how = "left", on =["LRD"])

merge_BatteryLifespan.drop_duplicates(subset=None, keep='first', inplace=True)
merge_BatteryLifespan.reset_index(drop = True, inplace= True)
merge_BatteryLifespan.dropna(inplace=True)

# group the battery backup hours
temp = {
     '<15min' : 6, '>15min to <30min' : 5, '>30min to <1hr' : 4,'>1hr to <2hrs' : 3,'>2hrs to <3hrs' : 2, '>3hrs to <4hrs' : 1, '>4hrs' : 0
}
merge_BatteryLifespan.replace(temp, inplace = True) 
merge_BatteryLifespan


,LRD,Aging Days,Backup Hour,Power Event Count,DC_Admin_Status,DC_Brand,DC_Capacity,Total_Bank,DC_Technology,CS_Site_Type,Cooling_System
0,LRD2246,1774,4,0.500000,Active,Narada,150,2,VRLA,Vacant Land,Air Conditioner
1,LRD2274,932,2,1.000000,Active,Huawei,150,3,Lithium,Rooftop,FCU Hybrid
2,LRD3701,2105,1,0.076923,Active,MSB,100,1,VRLA,Vacant Land,HEX
3,LRD3701,2105,2,0.076923,Active,MSB,100,1,VRLA,Vacant Land,HEX
4,LRD3701,2105,0,0.076923,Active,MSB,100,1,VRLA,Vacant Land,HEX
...,...,...,...,...,...,...,...,...,...,...,...
1632,LRD22396,1741,6,1.000000,Active,Enersys,150,2,VRLA,Rooftop,FCU Hybrid
1633,LRD3779,480,1,0.250000,Active,Sacredsun,150,1,VRLA,Vacant Land,HEX
1634,LRD3779,480,2,0.250000,Active,Sacredsun,150,1,VRLA,Vacant Land,HEX
1635,LRD3779,480,3,0.250000,Active,Sacredsun,150,1,VRLA,Vacant Land,HEX


Remove aging days outliers

In [120]:
# Remove the aging day outliers that are less than 700 and greater than 2100
merge_BatteryLifespan = merge_BatteryLifespan[merge_BatteryLifespan['Aging Days'] <= 2100]
merge_BatteryLifespan = merge_BatteryLifespan[merge_BatteryLifespan['Aging Days'] >= 700]
merge_BatteryLifespan.reset_index(drop = True, inplace = True)
merge_BatteryLifespan

,LRD,Aging Days,Backup Hour,Power Event Count,DC_Admin_Status,DC_Brand,DC_Capacity,Total_Bank,DC_Technology,CS_Site_Type,Cooling_System
0,LRD2246,1774,4,0.500000,Active,Narada,150,2,VRLA,Vacant Land,Air Conditioner
1,LRD2274,932,2,1.000000,Active,Huawei,150,3,Lithium,Rooftop,FCU Hybrid
2,LRD3749,1524,5,0.200000,Active,Narada,150,1,VRLA,Vacant Land,Air Conditioner
3,LRD3749,1524,6,0.200000,Active,Narada,150,1,VRLA,Vacant Land,Air Conditioner
4,LRD3921,1117,0,1.000000,Active,Enersys,150,2,VRLA,Vacant Land,HEX
...,...,...,...,...,...,...,...,...,...,...,...
1471,LRD15522,1188,1,1.000000,Active,Narada,150,4,VRLA,Vacant Land,FCU Hybrid
1472,LRD16749,1748,2,1.000000,Active,Enersys,100,2,VRLA,Vacant Land,HEX
1473,LRD19243,1420,2,0.333333,Active,Narada,100,2,VRLA,Vacant Land,HEX
1474,LRD19243,1420,3,0.333333,Active,Narada,100,2,VRLA,Vacant Land,HEX


Label Encoding the DC_Admin_Status and DC_Technology

In [121]:
# Use label encoding for DC_Admin_Status('Active', 'Decomissioned') and DC_Technology('VLRA', 'Lithium') because they have binary values
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

merge_BatteryLifespan['DC_Admin_Status'] = label_encoder.fit_transform(merge_BatteryLifespan['DC_Admin_Status']) 
merge_BatteryLifespan['DC_Technology'] = label_encoder.fit_transform(merge_BatteryLifespan['DC_Technology']) 
merge_BatteryLifespan.head

<bound method NDFrame.head of            LRD  Aging Days  Backup Hour  Power Event Count  DC_Admin_Status  \
0      LRD2246        1774            4           0.500000                0   
1      LRD2274         932            2           1.000000                0   
2      LRD3749        1524            5           0.200000                0   
3      LRD3749        1524            6           0.200000                0   
4      LRD3921        1117            0           1.000000                0   
...        ...         ...          ...                ...              ...   
1471  LRD15522        1188            1           1.000000                0   
1472  LRD16749        1748            2           1.000000                0   
1473  LRD19243        1420            2           0.333333                0   
1474  LRD19243        1420            3           0.333333                0   
1475  LRD22396        1741            6           1.000000                0   

     DC_Brand DC_Capa

Remove DC_Brand outliers

In [122]:
# Find for the battery brands that are outliers
merge_BatteryLifespan.groupby('DC_Brand')['DC_Brand'].size().sort_values(ascending = True)

DC_Brand
Haze           1
Sunstone       2
Supersave      2
Sacredsun     11
MSB           53
Huawei       101
Enersys      423
Narada       883
Name: DC_Brand, dtype: int64

In [123]:
# Remove the battery brand outliers
lst = ['Haze', 'Sunstone', 'Supersave', 'Sacredsun']
merge_BatteryLifespan = merge_BatteryLifespan[~merge_BatteryLifespan['DC_Brand'].isin(lst)]
merge_BatteryLifespan

,LRD,Aging Days,Backup Hour,Power Event Count,DC_Admin_Status,DC_Brand,DC_Capacity,Total_Bank,DC_Technology,CS_Site_Type,Cooling_System
0,LRD2246,1774,4,0.500000,0,Narada,150,2,1,Vacant Land,Air Conditioner
1,LRD2274,932,2,1.000000,0,Huawei,150,3,0,Rooftop,FCU Hybrid
2,LRD3749,1524,5,0.200000,0,Narada,150,1,1,Vacant Land,Air Conditioner
3,LRD3749,1524,6,0.200000,0,Narada,150,1,1,Vacant Land,Air Conditioner
4,LRD3921,1117,0,1.000000,0,Enersys,150,2,1,Vacant Land,HEX
...,...,...,...,...,...,...,...,...,...,...,...
1471,LRD15522,1188,1,1.000000,0,Narada,150,4,1,Vacant Land,FCU Hybrid
1472,LRD16749,1748,2,1.000000,0,Enersys,100,2,1,Vacant Land,HEX
1473,LRD19243,1420,2,0.333333,0,Narada,100,2,1,Vacant Land,HEX
1474,LRD19243,1420,3,0.333333,0,Narada,100,2,1,Vacant Land,HEX


Remove DC_Capacity outliers

In [124]:
# Find for the battery capacity outliers
merge_BatteryLifespan.groupby('DC_Capacity')['DC_Capacity'].size().sort_values(ascending = True)

DC_Capacity
155      43
100     187
150    1230
Name: DC_Capacity, dtype: int64

In [125]:
# Remove the battery capacity outliers
merge_BatteryLifespan = merge_BatteryLifespan[merge_BatteryLifespan['DC_Capacity'] != '200']
merge_BatteryLifespan

,LRD,Aging Days,Backup Hour,Power Event Count,DC_Admin_Status,DC_Brand,DC_Capacity,Total_Bank,DC_Technology,CS_Site_Type,Cooling_System
0,LRD2246,1774,4,0.500000,0,Narada,150,2,1,Vacant Land,Air Conditioner
1,LRD2274,932,2,1.000000,0,Huawei,150,3,0,Rooftop,FCU Hybrid
2,LRD3749,1524,5,0.200000,0,Narada,150,1,1,Vacant Land,Air Conditioner
3,LRD3749,1524,6,0.200000,0,Narada,150,1,1,Vacant Land,Air Conditioner
4,LRD3921,1117,0,1.000000,0,Enersys,150,2,1,Vacant Land,HEX
...,...,...,...,...,...,...,...,...,...,...,...
1471,LRD15522,1188,1,1.000000,0,Narada,150,4,1,Vacant Land,FCU Hybrid
1472,LRD16749,1748,2,1.000000,0,Enersys,100,2,1,Vacant Land,HEX
1473,LRD19243,1420,2,0.333333,0,Narada,100,2,1,Vacant Land,HEX
1474,LRD19243,1420,3,0.333333,0,Narada,100,2,1,Vacant Land,HEX


Remove Total_Bank outliers

In [126]:
# Find for the total bank outliers
merge_BatteryLifespan.groupby('Total_Bank')['Total_Bank'].size().sort_values(ascending = True)

Total_Bank
6      2
5      7
4     24
3    139
2    531
1    757
Name: Total_Bank, dtype: int64

In [127]:
# Remove the total bank outliers
lst = ['5', '6']
merge_BatteryLifespan = merge_BatteryLifespan[~merge_BatteryLifespan['Total_Bank'].isin(lst)]
merge_BatteryLifespan

,LRD,Aging Days,Backup Hour,Power Event Count,DC_Admin_Status,DC_Brand,DC_Capacity,Total_Bank,DC_Technology,CS_Site_Type,Cooling_System
0,LRD2246,1774,4,0.500000,0,Narada,150,2,1,Vacant Land,Air Conditioner
1,LRD2274,932,2,1.000000,0,Huawei,150,3,0,Rooftop,FCU Hybrid
2,LRD3749,1524,5,0.200000,0,Narada,150,1,1,Vacant Land,Air Conditioner
3,LRD3749,1524,6,0.200000,0,Narada,150,1,1,Vacant Land,Air Conditioner
4,LRD3921,1117,0,1.000000,0,Enersys,150,2,1,Vacant Land,HEX
...,...,...,...,...,...,...,...,...,...,...,...
1471,LRD15522,1188,1,1.000000,0,Narada,150,4,1,Vacant Land,FCU Hybrid
1472,LRD16749,1748,2,1.000000,0,Enersys,100,2,1,Vacant Land,HEX
1473,LRD19243,1420,2,0.333333,0,Narada,100,2,1,Vacant Land,HEX
1474,LRD19243,1420,3,0.333333,0,Narada,100,2,1,Vacant Land,HEX


Perform one-hot encoding

In [128]:
# Perform one-hot encoding for DC_Brand, Cooling_System, CS_Site_Type, and DC_Capacity
from sklearn.preprocessing import OneHotEncoder
merge_BatteryLifespan.reset_index(drop = True, inplace = True)
ohe = OneHotEncoder(sparse = False)

cat_column = pd.DataFrame(merge_BatteryLifespan['DC_Brand'])
result = ohe.fit_transform(cat_column)
dc_brand_enc = pd.DataFrame(result, columns = ohe.categories_)

cat_column = pd.DataFrame(merge_BatteryLifespan['Cooling_System'])
result = ohe.fit_transform(cat_column)
cs_enc = pd.DataFrame(result, columns = ohe.categories_)

cat_column = pd.DataFrame(merge_BatteryLifespan['CS_Site_Type'])
result = ohe.fit_transform(cat_column)
cs_site_type_enc = pd.DataFrame(result, columns = ohe.categories_)

cat_column = pd.DataFrame(merge_BatteryLifespan['DC_Capacity'])
result = ohe.fit_transform(cat_column)
dc_capacity_enc = pd.DataFrame(result, columns = ohe.categories_)

encoded = merge_BatteryLifespan.join(dc_brand_enc)
encoded = encoded.join(cs_enc)
encoded = encoded.join(cs_site_type_enc)
encoded = encoded.join(dc_capacity_enc)
encoded.drop(['DC_Brand', 'Cooling_System', 'CS_Site_Type', 'DC_Capacity'], axis = 1, inplace = True)
encoded

,LRD,Aging Days,Backup Hour,Power Event Count,DC_Admin_Status,Total_Bank,DC_Technology,"(Enersys,)","(Huawei,)","(MSB,)",...,"(Air Conditioner,)","(FCU,)","(FCU Hybrid,)","(HEX,)","(Inbuilding,)","(Rooftop,)","(Vacant Land,)","(100,)","(150,)","(155,)"
0,LRD2246,1774,4,0.500000,0,2,1,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,LRD2274,932,2,1.000000,0,3,0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,LRD3749,1524,5,0.200000,0,1,1,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,LRD3749,1524,6,0.200000,0,1,1,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,LRD3921,1117,0,1.000000,0,2,1,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446,LRD15522,1188,1,1.000000,0,4,1,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1447,LRD16749,1748,2,1.000000,0,2,1,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
1448,LRD19243,1420,2,0.333333,0,2,1,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
1449,LRD19243,1420,3,0.333333,0,2,1,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0


**Create the S3 bucket**

In [129]:
bucket_name = 'maxis-problem'
my_region = boto3.session.Session().region_name
s3 = boto3.resource('s3')

try:
    if my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucket_name)
    else:
        s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': my_region})
    print('S3 bucket created successfully')
except Exception as e:
    print ('S3 errpr: ', e)

S3 errpr:  An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


In [130]:
#save merged_BatteryLifespan to bucket
to_export = merge_BatteryLifespan
to_export.to_csv('Merged-Dataset-1.csv', index=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'problem-1/Merged-Dataset-1.csv')).upload_file('Merged-Dataset-1.csv')
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')
